In [1]:
from datetime import timedelta
import re

class SubtitleCorrector:
    def __init__(self, input_path):
        self.input_path = input_path
        self.timestamp_pattern = re.compile(r'(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})')

    def parse_timestamp(self, timestamp):
        hours, minutes, seconds_millis = timestamp.split(':')
        seconds, millis = seconds_millis.split(',')
        return timedelta(
            hours=int(hours), minutes=int(minutes), seconds=int(seconds), milliseconds=int(millis)
        )

    def format_timestamp(self, td):
        total_seconds = int(td.total_seconds())
        millis = int(td.microseconds / 1000)
        hours = total_seconds // 3600
        minutes = (total_seconds % 3600) // 60
        seconds = total_seconds % 60
        return f"{hours:02}:{minutes:02}:{seconds:02},{millis:03}"

    def corrector(self,hours=0, minutes=0, seconds=0, milliseconds=0, direction='subtract', output_path='output.srt'):
        """
        Adjusts subtitle timestamps by adding or subtracting the specified time.
        """
        delta = timedelta(hours=hours, minutes=minutes, seconds=seconds, milliseconds=milliseconds)
        if direction not in ('add', 'subtract'):
            raise ValueError("direction must be 'add' or 'subtract'")

        with open(self.input_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        new_lines = []
        for line in lines:
            match = self.timestamp_pattern.match(line.strip())
            if match:
                start, end = match.groups()
                start_td = self.parse_timestamp(start)
                end_td = self.parse_timestamp(end)
                if direction == 'add':
                    start_td += delta
                    end_td += delta
                else:
                    start_td -= delta
                    end_td -= delta
                # Clamp to zero if negative
                if start_td.total_seconds() < 0:
                    start_td = timedelta(0)
                if end_td.total_seconds() < 0:
                    end_td = timedelta(0)
                new_line = f"{self.format_timestamp(start_td)} --> {self.format_timestamp(end_td)}\n"
                new_lines.append(new_line)
            else:
                new_lines.append(line)

        with open(output_path, 'w', encoding='utf-8') as f:
            f.writelines(new_lines)
        return output_path

In [4]:
sc = SubtitleCorrector('sample_data/sample_subtitle.srt')
sc.corrector(seconds=10,milliseconds=500, direction='add', output_path='sample_data/output.srt')

'sample_data/output.srt'